# Demonstration of Python REST API for Thingsboard

This little demo shows the basic functionalities provided by the thingsboard REST Api. We can use it to 

- create / delete users
- fetch latest telemetry
- update dashboards
- etc...

In [47]:
import logging
from json import load
# Importing models and REST client class from Professional Edition version
from tb_rest_client.rest_client_pe import *
from tb_rest_client.rest import ApiException
from datetime import datetime


def format_datetime(dt_obj: datetime):
    y,m,d,h,minu, *_ = dt_obj.timetuple()
    return "%s-%s-%s  %s:%s" % (y,m,d,h,minu)

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(module)s - %(lineno)d - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


# ThingsBoard REST API URL
url = ...
LEUPHANA_TENANT_ID = ...
TEST_DEVICE_ID = ...
ROBO_TOKEN = ...

In [48]:
with open("secrets.json", "r") as user_file:
    user = load(user_file)

In [49]:
admin = user["admin"]
tenant = user["tenant"]

## Initialize REST Client

In [50]:
admin_client = RestClientPE(base_url=url)
admin_client.login(username=admin["mail"], password=admin["pw"])
print("User is of type %s" % admin_client.get_user().authority)

User is of type SYS_ADMIN


In [51]:
tenant_client = RestClientPE(base_url=url)
tenant_client.login(username=tenant["mail"], password=tenant["pw"])
print("User is of type %s" % tenant_client.get_user().authority)

User is of type TENANT_ADMIN


### Get names of all Leuphana Admin Users

In [52]:
[x.first_name for x in 
 admin_client.get_tenant_admins(LEUPHANA_TENANT_ID, 10, 0).data]

['Laurin', 'Kenneth', 'Ole Christian']

### Add new Telemetry to our Test Device

In [55]:
tenant_client.telemetry_controller.get_timeseries_keys_using_get1("DEVICE", ROBO_TOKEN)

['latitude',
 'longitude',
 'temperature',
 'speed',
 'num_tasks',
 'batteryLevel',
 'battery']

0.0002

In [65]:
tenant_client.asset_controller.get_asset_info_by_id_using_get(TEST_DEVICE_ID)

{'additional_info': {'description': ''},
 'asset_profile_id': {'entity_type': 'ASSET_PROFILE',
                      'id': '81850a30-5592-11ee-a932-412d54b899cb'},
 'asset_profile_name': 'Task',
 'created_time': 1694958987865,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '13814000-1dd2-11b2-8080-808080808080'},
 'customer_is_public': False,
 'customer_title': None,
 'id': {'entity_type': 'ASSET', 'id': 'ff4f7490-5561-11ee-a932-412d54b899cb'},
 'label': 'Nächster Transportauftrag für Robi',
 'name': 'next_task',
 'tenant_id': {'entity_type': 'TENANT',
               'id': '796e2850-508c-11ee-a932-412d54b899cb'},
 'type': 'Task'}

In [66]:
tenant_client.device_controller.get_device_by_id_using_get(ROBO_TOKEN)

{'additional_info': {'description': '',
                     'gateway': False,
                     'overwriteActivityTime': False},
 'created_time': 1694725551141,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '13814000-1dd2-11b2-8080-808080808080'},
 'device_data': {'configuration': {'type': 'DEFAULT'},
                 'transport_configuration': {'type': 'DEFAULT'}},
 'device_profile_id': {'entity_type': 'DEVICE_PROFILE',
                       'id': '7975f080-508c-11ee-a932-412d54b899cb'},
 'firmware_id': None,
 'id': {'entity_type': 'DEVICE', 'id': '7c2e2550-5342-11ee-a932-412d54b899cb'},
 'label': 'robot',
 'name': 'Robot LL',
 'software_id': None,
 'tenant_id': {'entity_type': 'TENANT',
               'id': '796e2850-508c-11ee-a932-412d54b899cb'},
 'type': 'default'}

In [46]:
_ = tenant_client.telemetry_controller.save_entity_telemetry_using_post(
    entity_type="ASSET",
    entity_id=TEST_DEVICE_ID,
    scope="a",
    body={
        "longitude": 10.4034710,
        "latitude": 53.2280540
    }
)
print("Successfuly sent telemetry")

Successfuly sent telemetry


In [45]:
_ = tenant_client.telemetry_controller.save_entity_telemetry_using_post(
    entity_type="DEVICE",
    entity_id=ROBO_TOKEN,
    scope="a",
    body={
        "longitude": 10.4013610,
        "latitude": 53.2289540,
        "temperature": 54,
    }
)
print("Successfuly sent telemetry")

Successfuly sent telemetry


### Fetch the latest telemtry of the device. This way, we can track our devices programatically in our scripts

In [9]:
tel = tenant_client.telemetry_controller.get_latest_timeseries_using_get("DEVICE", TEST_DEVICE_ID)
timestamp = format_datetime(datetime.fromtimestamp(tel["temperature"][0]["ts"]  / 1000))
value = tel["temperature"][0]["value"]
print("Latest temeletry is %s and got posted on %s" % (value, timestamp))

Latest temeletry is 31 and got posted on 2023-9-17  21:14
